In [15]:
import os
import requests
import pandas as pd
import warnings
import psycopg2
import numpy as np
from datetime import datetime, timedelta
from get_spotify import *
from credentials import *
from sqlalchemy import create_engine

In [2]:
warnings.filterwarnings('ignore')
token = get_token()

In [3]:
df_albums_releases = get_albums_releases(token=token, delta_days=365)
df_albums = get_several_albums(token, list(df_albums_releases)[:20])

In [11]:
df_albums["spotify_id"] = df_albums["id"]
df_albums[["spotify_id", "name", "release_date", "genres", "label"]].head()

,id,name,release_date,genres,label,popularity,artists_ids,tracks_ids,spotify_id
0,51QJlvJclfVB6jNNXlDatS,Insomnia,2024-03-01,[],TOY'S FACTORY,53,[58oPVy7oihAEXE0Ott6JOf],[52KxsRBZj7Ip9HdxgvEQbO],51QJlvJclfVB6jNNXlDatS
1,6L3OPsn0P4F6PNUDFGxDVV,Get Lower,2024-02-09,[],Dim Mak Records,53,"[77AiFEVeAVj2ORpC85QVJs, 7sfl4Xt5KmfyDs2T3SVSMK]",[7xKmcNxk4wF03Xpc4oIXbg],6L3OPsn0P4F6PNUDFGxDVV
2,1DCiXnF1Z9XX4zxYCFytSX,Never Be Lonely,2024-02-09,[],Polydor Records,67,"[4Q6nIcaBED8qUel8bBx6Cr, 03d2mJXSMtuPI0nIvLnhoS]",[4beKzTMCCMDizozv4Y3Shj],1DCiXnF1Z9XX4zxYCFytSX
3,27LVMe5wMsHddairHoCux2,We Can't Stop,2024-02-09,[],Tomorrowland Music,56,"[4D75GcNG95ebPtNvoNVXhz, 0CbeG1224FS58EUx4tPev...",[7b5FO1uKhuJE0ZUJPKLWtI],27LVMe5wMsHddairHoCux2
4,0Tqz0PyiUMHr2HIwvXmyKP,crush,2024-02-09,[],Chosen People,39,[0duLKMlcwhyZgqu8zSSjBp],[3njA65KM3YEF08hEBGpaTW],0Tqz0PyiUMHr2HIwvXmyKP


In [12]:
df_albums["spotify_id"] = df_albums["id"]
df_albums = df_albums[["spotify_id", "name", "release_date", "genres", "label"]]
df_albums.head()

,spotify_id,name,release_date,genres,label
0,51QJlvJclfVB6jNNXlDatS,Insomnia,2024-03-01,[],TOY'S FACTORY
1,6L3OPsn0P4F6PNUDFGxDVV,Get Lower,2024-02-09,[],Dim Mak Records
2,1DCiXnF1Z9XX4zxYCFytSX,Never Be Lonely,2024-02-09,[],Polydor Records
3,27LVMe5wMsHddairHoCux2,We Can't Stop,2024-02-09,[],Tomorrowland Music
4,0Tqz0PyiUMHr2HIwvXmyKP,crush,2024-02-09,[],Chosen People


In [13]:
def insert_values(conn, df, table):

    tuples = [tuple(x) for x in df.to_numpy()]

    cols = ','.join(list(df.columns))
    # SQL query to execute
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        psycopg2.extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Error: {error}")
        conn.rollback()
        cursor.close()
        return 1
    print("the dataframe is inserted")
    cursor.close()

In [14]:
conn = psycopg2.connect( 
    database=os.getenv("PG_DATABASE"),
    user=os.getenv("PG_USERNAME"),
    password=os.getenv("PG_PASSWORD"),
    host=os.getenv("PG_HOST"),
    port=os.getenv("PG_PORT")
)

insert_values(conn=conn, df=df_albums, table="albums")

the dataframe is inserted


In [32]:
# get_several_artists.py

albums_ids = get_albums_releases(token)
df_albums = get_several_albums(token, albums_ids)
artists_ids = []
for alist in df_albums['artists_ids']:
    artists_ids = artists_ids + alist

headers = {
    "Authorization": f"Bearer {token}"
}
params = {
    "ids": ','.join(artists_ids)
}
response = requests.get(
    "https://api.spotify.com/v1/artists",
    headers=headers,
    params=params
)

df = pd.json_normalize(response.json()["artists"])
df_out = df[['id', 'name', 'genres', 'popularity']]

In [11]:
# get_several_tracks.py

albums_ids = get_albums_releases(token)
df_albums = get_several_albums(token, albums_ids)
tracks_ids = []
for tlist in df_albums['tracks_ids']:
    tracks_ids = tracks_ids + tlist

counter_from = 0
counter_to = min(49, len(tracks_ids))

df = pd.DataFrame()
for i in range(int(len(tracks_ids) / counter_to) + 1):
    # print(f'{counter_from}:{counter_to}')
    
    headers = {
        "Authorization": f"Bearer {token}"
    }
    params = {
        "ids": ','.join(tracks_ids[counter_from:counter_to])
    }
    response = requests.get(
        "https://api.spotify.com/v1/tracks",
        headers=headers,
        params=params
    )
    
    df_iter = pd.json_normalize(response.json()["tracks"])
    df = pd.concat([df, df_iter], ignore_index=True)
    
    counter_from += 50
    counter_to += min(50, len(tracks_ids) - counter_to)

df_tracks = df[["id", "name", "album.id", "duration_ms", "explicit", "popularity"]]
df_tracks["artists_ids"] = df["artists"].apply(lambda artists_data: [artist["id"] for artist in artists_data])

df_tracks.head()

,id,name,album.id,duration_ms,explicit,popularity,artists_ids
0,3eP13S8D5m2cweMEg3ZDed,Virginia Beach,4czdORdCWP9umpbhFXK2fW,251094,True,79,[3TVXtAsR1Inumwj472S9r4]
1,0Mrnt1YqVuW2bqmwu4VxDt,Amen (feat. Teezo Touchdown),4czdORdCWP9umpbhFXK2fW,141212,True,77,"[3TVXtAsR1Inumwj472S9r4, 0fGcIStdT1OpFFhOC7Wp36]"
2,2nibvvDdAQkVraYP00z2RS,Calling For You (feat. 21 Savage),4czdORdCWP9umpbhFXK2fW,285630,True,78,"[3TVXtAsR1Inumwj472S9r4, 1URnnhqYAYcrqrcwql10ft]"
3,53KaP77tkliz36LPxWGlWK,Fear Of Heights,4czdORdCWP9umpbhFXK2fW,155154,True,78,[3TVXtAsR1Inumwj472S9r4]
4,4ZT0wGKwlJRjpMUuwZVy3p,Daylight,4czdORdCWP9umpbhFXK2fW,164142,True,78,[3TVXtAsR1Inumwj472S9r4]


In [28]:
def iterate(num_tracks: int):
    counter_from = 0
    counter_to = min(50, num_tracks)

    for i in range(int(num_tracks / counter_to) + 1):
        print(f'{counter_from}:{counter_to}')
        counter_from += 50
        counter_to += min(50, num_tracks - counter_to)

iterate(len(tracks_ids))

0:50
50:100
100:150
150:165
